## Pipeline

Tot nu toe is de code voor het opbouwen van een model vespreid over twee stappen. 
Een preprocesing stap waar alle transformaties op gebeuren op de data bruikbaar te maken zoals schaling, invullen van ontbrekende waarden, hogere orde features, ...
In een latere stap gebeurd dan het trainen van een model.
Echter is het zo dat als je het model in gebruik wilt nemen in een applicatie dat je ook exact dezelfde preprocessing stappen nodig hebt omdat je de data die je aan het model presenteert hetzelfde moet zijn.
Om deze reden is de vorige aanpak niet praktisch aangezien het eenvoudig is om een fout te maken.
Het is ook geen onderhoudbare oplossing omdat het veel manueel werk vereist indien er een bepaalde stap aangepakt wordt.

Om dit tegen te gaan kan men gebruik maken van Pipelines. 
Hiermee definieer je de flow die de data moet ondergaan om verwerkt te worden. 
De volledige pipeline of flow kan dan gebruikt worden om te trainen en te gebruiken in productie. 
Hierdoor worden de preprocessing stappen (zoals scalers, ordinal of one-hotencoder) steeds enkel op de training folds uitgevoerd.
Een bijkomende voordeel is dat ook parameters van de preprocessing stappen meegenomen kunnen worden in de gridsearch.

Let op dat NaN waarden invullen kan gebeuren in de pipeline maar rijen weglaten kan niet gedaan worden tijdens de pipeline. Echter is het gebruik van een pipeline een goede stap in het verhogen van de leesbaarheid van een code stuk en wordt het daardoor veelvuldig gebruikt.

In [2]:
import numpy as np

from sklearn.compose import ColumnTransformer
from sklearn.datasets import fetch_openml
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, GridSearchCV

np.random.seed(0)

X, y = fetch_openml("titanic", version=1, as_frame=True, return_X_y=True)
X

,pclass,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,boat,body,home.dest
0,1,"Allen, Miss. Elisabeth Walton",female,29.0000,0,0,24160,211.3375,B5,S,2,NaN,"St Louis, MO"
1,1,"Allison, Master. Hudson Trevor",male,0.9167,1,2,113781,151.5500,C22 C26,S,11,NaN,"Montreal, PQ / Chesterville, ON"
2,1,"Allison, Miss. Helen Loraine",female,2.0000,1,2,113781,151.5500,C22 C26,S,NaN,NaN,"Montreal, PQ / Chesterville, ON"
3,1,"Allison, Mr. Hudson Joshua Creighton",male,30.0000,1,2,113781,151.5500,C22 C26,S,NaN,135.0,"Montreal, PQ / Chesterville, ON"
4,1,"Allison, Mrs. Hudson J C (Bessie Waldo Daniels)",female,25.0000,1,2,113781,151.5500,C22 C26,S,NaN,NaN,"Montreal, PQ / Chesterville, ON"
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1304,3,"Zabour, Miss. Hileni",female,14.5000,1,0,2665,14.4542,NaN,C,NaN,328.0,NaN
1305,3,"Zabour, Miss. Thamine",female,NaN,1,0,2665,14.4542,NaN,C,NaN,NaN,NaN
1306,3,"Zakarian, Mr. Mapriededer",male,26.5000,0,0,2656,7.2250,NaN,C,NaN,304.0,NaN
1307,3,"Zakarian, Mr. Ortin",male,27.0000,0,0,2670,7.2250,NaN,C,NaN,NaN,NaN


In [11]:
# stap 1: welke soorten kolommen zijn er
numeric_features = ['age', 'fare']
do_nothing_features = ['pclass', 'sibsp', 'parch']
one_hot_features = ['sex', 'embarked']
ordinal_features = ['home.dest']

# stap 2: preprocessor
numeric_pipeline = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value=0)),
    ('scaler', StandardScaler())
])
ordinal_pipeline = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='unknown')),
    ('scaler', OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1))
])

preprocessor = ColumnTransformer(transformers=[
    ('numeric', numeric_pipeline, numeric_features),
    ('nothings', SimpleImputer(), do_nothing_features),
    ('onehots', OneHotEncoder(), one_hot_features),
    ('ordinals', ordinal_pipeline, ordinal_features)
])

# stap 3: pipeline
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('clf', LogisticRegression())
])

# stap 4 train het model
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

pipeline.fit(X_train, y_train)
pipeline.score(X_test, y_test)

c:\Users\jens.baetens3\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.7862595419847328

In [ ]:
params = {
    'preprocessor__numeric__imputer__fill_value': [0, -1, 1], # dubbele underscore tussen niveaus (hyperparameters hebben vaak een underscore in de naam)
    'clf__C': np.arange(1,10)
}

grid_search = GridSearchCV(pipeline, param_grid=params)

## Oefening

Maak een pipeline voor een gridsearch uit te voeren op de diabetes-dataset in sklearn. Gebruik een Min-Max scaler voor de numerieke waarden en een ordinal encoder voor de categorieke kolommen.
Zoek naar de beste combinatie van hyperparameters van een SVM-classifier door middel van een grid search met cross validation.